In [1]:
import pandas as pd
import requests
from urllib.request import Request, urlopen
import re
from bs4 import BeautifulSoup
import wget
import os
import sys
import win32com.client as win32
from win32com.client import Dispatch

In [ ]:
#Página de origem dos links - a ser raspada
url = 'https://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/index.php?p=269652'

#Fazer a requisição 
req = requests.get(url)
req = req.text

#aplicar o BS a requisição
bs = BeautifulSoup(req, "html.parser") #print(bs.prettify())

In [ ]:
#Limpar o conteúdo da requisição: localiar tag a / classe calendários / e a tag a novamente dentro de calendário
# avaliar se é possível simplificar esse processo.
base = bs.find_all(class_='calendarios')
for link in base:
    base = link.find_all('a')

#extrair os links
base_aux = []
for href in base:
    base_aux.append(href.get('href'))
    
#consolidar somente os links em um DF
for item in base_aux:
    if 'Consolidado' in item:
        base_aux.remove(item)
base = pd.DataFrame(base_aux)

#renomear a coluna
base.columns = ['link']

In [ ]:
##BAIXAR PLANILHAS DO SITE
# Esse passo foi necessario porque a planilha contem um resíduo/sujeira de macro, 
# que so aparece quando o arquivo é aberto pela primeira vez. 
# Inicialmente pretendia utilizar o urllib mas gerava um erro. 
#Somente com essa alternativa e a conversão do XLS para XLSX foi possível contornar o problema.

for index, row in base.iterrows():
    url_dia = row['link']
    wget.download(url_dia, 'E:/user/2019inicio/')

In [3]:
##CONVERTER PLANILHAS DE XLS PARA XLSX

#pasta inicial onde estao os arquivos
dirname = r'E:/user/2019inicio'
#pasta onde serao salvos os arquivos
dirname_final = r'E:/user/2019final' 

In [ ]:
#Caso tenha problemas com win32, uma referencia para consertar o problema
#https://stackoverflow.com/questions/52889704/python-win32com-excel-com-model-started-generating-errors

In [ ]:
#Monta a aplicação para abir o Excel
excel = win32.gencache.EnsureDispatch('Excel.Application')
excel.DisplayAlerts = False
excel.EnableEvents = False

#Itera cada planilha no diretório onde os arquivos foram salvos
for planilha in os.listdir(dirname):
    
    #Decode para permitir abrir o arquivo no proximo passo. Sem isso, estava tendo problema com o nome do arquivo
    filename = os.fsdecode(planilha) 
    
    #Utiliza a aplicação para abrir o arquivo, e duas configurações paa evitar prompt de mensagens
    wb = excel.Workbooks.Open(dirname + '\\' + filename)
    wb.DoNotPromptForConvert = True
    wb.CheckCompatibility = False

    #salva o conteudo da celula A1 como a variável nome
    nome = str(wb.Sheets[1].Cells(4,1).Value)
    nome = nome[:10]
    
    #deleta as outras abas da planilha
    num = wb.Sheets.Count
    if num >= 2:
        wb.Sheets[2].Delete()
            
    #salva o arquivo em formato xlsx - 51, com a variavel nome, no diretório final
    wb.SaveAs(dirname_final + '\\' + nome, FileFormat = 51)

    wb.Close(SaveChanges=True)

#desmota a Aplicacao
excel.Application.Quit()
del excel
excel = None

In [4]:
geral = pd.DataFrame()

#Iterando em cada planilha
for planilha_final in os.listdir(dirname_final):
    filename = os.fsdecode(planilha_final)     
    base = pd.read_excel(dirname_final + '\\' + filename, header=None)
    
    #Somente serão consolidados as 5 primeiras colunas e a última da planilha, que contem o total de passageiros.
    #foi uma solução mais simples, porque as planilhas mudaram de formato ao longo do ano e fazer a correspondencias entre 
    # os dois modelos iria dar muito trabalho adicional
    fim = len(base.columns) - 1
    base_1 = base.iloc[:, [0, 1, 2, 3, 4, fim]]
    base_1.columns = ['data', 'tipo', 'area', 'empresa', 'linha', 'passageiros']
    
    geral = pd.concat([geral, base_1])

In [6]:
#Como as planilhas original possuem uma primeira linha descritiva é preciso elimina-la. 
#E como o formato pode variar, não é recomendado fazer pela posição
#Também elimina a linha em branco entre a primeira linha e o cabeçalho da tabela
geral.dropna(axis=0, thresh=5, inplace=True)

#Remover a cabeçalho original da tabela da planilha
geral = geral[geral.linha != 'Linha']

In [7]:
#Ajustado o campo de data
geral['data'] = geral['data'].astype(str)

#Corrigindo uma data onde consta 2018
geral['data'] = geral['data'].str.replace('2018', '2019')

#Convertendo em data
geral['data'] = pd.to_datetime(geral['data'], dayfirst=True)

In [15]:
#salvar em um csv
geral.to_csv('base_2019.csv', index=False)